In [1]:
# Install PaddleOCR and necessary libraries
#!pip install paddlepaddle paddleocr


In [2]:
import cv2
from paddleocr import PaddleOCR
import numpy as np
import csv
import tensorflow as tf


In [3]:
# Initialize PaddleOCR
ocr = PaddleOCR(lang='en')

# Define the image path
image_dir = "FoodLabels"
image_path = f"{image_dir}/Image_CornFlakes.png"

# Load the image
image = cv2.imread(image_path)


[2024/11/16 21:50:30] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\ajayr/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\ajayr/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [ ]:
# Perform OCR on the image
ocr_results = ocr.ocr(image_path)

print(ocr_results)


[2024/11/16 21:50:32] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2024/11/16 21:50:33] ppocr DEBUG: dt_boxes num : 78, elapsed : 0.8747687339782715


In [ ]:
# Extract Bounding Boxes
boxes=[]
texts=[]
probabilities=[]
for line in ocr_results:
    # print(line)
    for i in line:
        boxes.append(i[0])
        texts.append(i[1][0])
        probabilities.append(i[1][1])

print(boxes)
print(texts)
print(probabilities)


[[[6.0, 10.0], [60.0, 7.0], [61.0, 30.0], [7.0, 33.0]], [[227.0, 11.0], [295.0, 8.0], [295.0, 35.0], [228.0, 37.0]], [[322.0, 12.0], [352.0, 12.0], [352.0, 39.0], [322.0, 39.0]], [[403.0, 12.0], [481.0, 9.0], [482.0, 36.0], [404.0, 38.0]], [[8.0, 36.0], [123.0, 36.0], [123.0, 56.0], [8.0, 56.0]], [[244.0, 37.0], [294.0, 37.0], [294.0, 60.0], [244.0, 60.0]], [[415.0, 38.0], [479.0, 38.0], [479.0, 60.0], [415.0, 60.0]], [[7.0, 66.0], [74.0, 66.0], [74.0, 87.0], [7.0, 87.0]], [[258.0, 70.0], [295.0, 70.0], [295.0, 94.0], [258.0, 94.0]], [[315.0, 68.0], [358.0, 68.0], [358.0, 95.0], [315.0, 95.0]], [[439.0, 68.0], [482.0, 72.0], [479.0, 99.0], [436.0, 95.0]], [[8.0, 91.0], [163.0, 91.0], [163.0, 112.0], [8.0, 112.0]], [[258.0, 93.0], [297.0, 96.0], [295.0, 119.0], [257.0, 116.0]], [[316.0, 93.0], [356.0, 93.0], [356.0, 118.0], [316.0, 118.0]], [[8.0, 115.0], [220.0, 115.0], [220.0, 135.0], [8.0, 135.0]], [[258.0, 116.0], [298.0, 119.0], [296.0, 147.0], [256.0, 144.0]], [[9.0, 141.0], [210.

In [ ]:
# Helper function to check if boxes are aligned
def are_boxes_aligned(box1, box2, axis="horizontal", threshold=10):
    """Check if two boxes are aligned either horizontally or vertically."""
    if axis == "horizontal":
        return abs(box1[0][1] - box2[0][1]) < threshold
    elif axis == "vertical":
        return abs(box1[0][0] - box2[0][0]) < threshold


In [ ]:
# Detect tables based on alignment
table_candidates = []
used_boxes = set()

In [ ]:
for i, box1 in enumerate(boxes):
    row_group = [i]  # Initialize a new group
    for j, box2 in enumerate(boxes):
        if j != i and are_boxes_aligned(box1, box2, axis="horizontal"):
            row_group.append(j)
    if len(row_group) > 1:  # Only consider groups with more than one element
        table_candidates.append(row_group)
        used_boxes.update(row_group)

In [ ]:
print(row_group)

[77, 75, 76]


In [ ]:
# Save the image with detected table regions
cv2.imwrite("detected_tables.jpg", image)

True

In [ ]:
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]

In [ ]:
im = image_cv.copy()

In [ ]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([int(x_h),int(y_h),int(x_h+width_h),int(y_h+height_h)])
  vert_boxes.append([int(x_v),int(y_v),int(x_v+width_v),int(y_v+height_v)])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)


In [ ]:
cv2.imwrite('horiz_vert.jpg',im)

True

In [ ]:
horiz_boxes = [[int(value) for value in row] for row in horiz_boxes]
print(horiz_boxes)
print(probabilities)

[[0, 10, 530, 30], [0, 11, 530, 35], [0, 12, 530, 39], [0, 12, 530, 36], [0, 36, 530, 56], [0, 37, 530, 60], [0, 38, 530, 60], [0, 66, 530, 87], [0, 70, 530, 94], [0, 68, 530, 95], [0, 68, 530, 99], [0, 91, 530, 112], [0, 93, 530, 119], [0, 93, 530, 118], [0, 115, 530, 135], [0, 116, 530, 147], [0, 141, 530, 159], [0, 140, 530, 172], [0, 166, 530, 183], [0, 165, 530, 194], [0, 166, 530, 193], [0, 200, 530, 221], [0, 196, 530, 224], [0, 230, 530, 247], [0, 224, 530, 250], [0, 225, 530, 255], [0, 251, 530, 269], [0, 242, 530, 276], [0, 272, 530, 292], [0, 269, 530, 293], [0, 266, 530, 293], [0, 266, 530, 297], [0, 294, 530, 312], [0, 288, 530, 316], [0, 289, 530, 318], [0, 319, 530, 341], [0, 315, 530, 343], [0, 316, 530, 343], [0, 346, 530, 369], [0, 345, 530, 368], [0, 344, 530, 369], [0, 344, 530, 369], [0, 372, 530, 394], [0, 393, 530, 414], [0, 389, 530, 417], [0, 393, 530, 418], [0, 397, 530, 420], [0, 421, 530, 443], [0, 421, 530, 447], [0, 424, 530, 447], [0, 425, 530, 449], [0, 

In [ ]:
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [ ]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

[ 0  6  7 13 14 16 19 21 24 26 29 34 35 38 42 46 50 54 58 62 66 70 72 77]


In [ ]:
im_nms = image_cv.copy()

In [ ]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)


In [ ]:
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [ ]:
print(vert_out)

tf.Tensor([70 67 13 72 69], shape=(5,), dtype=int32)


In [ ]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

[13 67 69 70 72]


In [ ]:
cv2.imwrite('im_nms.jpg',im_nms)

True

In [ ]:
out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

(24, 5)
[['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', '']]


In [ ]:
unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

[316, 0, 356, 656]
[7, 0, 56, 656]
[382, 0, 436, 656]
[462, 0, 497, 656]
[244, 0, 294, 656]


In [ ]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

[1 4 0 2 3]


In [ ]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [ ]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [ ]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

In [ ]:
out_array=np.array(out_array)

In [ ]:
out_array

array([['ENERGY', '378 kcal', '6%', '260kcal', '260kcal'],
       ['ENERGY FROM FAT', '9kcal', '', '84kcal', '84kcal'],
       ['TOTALFAT', '1.0g', '0.4%', '', '9.3g'],
       ['SATURATED FATTY ACIDS', '0.6g', '0.8%', '', ''],
       ['MONOUNSATURATED FATTY ACIDS', '0.1g', '', '', ''],
       ['POLYUNSATURATED FATTY ACIDS', '0.2g', '', '', ''],
       ['TRANS FATTY ACIDS', 'og', '0%', '', ''],
       ['CHOLESTEROL', 'omg', '', '', ''],
       ['TOTAL CARBOHYDRATES', '9.2g', '', '', '36.0 g'],
       ['OFWHICH TOTAL SUGARS', '9.2g', '', '', ''],
       ['OF WHICH ADDED SUGARS', '8.0g', '5%', '', '2.4g'],
       ['OFWHICH DIETARY FIBRE', '2.5g', '', '', '0.8g'],
       ['PROTEIN', '6.7g', '', '', '8.5 g'],
       ['SODIUM', '490 mg', '7%', '207 mg', '207 mg'],
       ['', '33.0 mg', '', '', '%RDA'],
       ['VITAMINC', '33.0 mg', '', '13.9 mg', '21%'],
       ['VITAMIN B1', '1.0 mg', '', '0.4 mg', '26%'],
       ['VITAMIN B2', '1.2 mg', '', '0.6 mg', '30%'],
       ['VITAMIN B3', '13.4 m

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(out_array).to_csv('Final_Table.csv')